<a href="https://colab.research.google.com/github/lechemrc/Datasets-to-ref/blob/master/Pitchfork%20Reviews/Pitchfork_Reviews_Data_Project_LS_Unit_2_merging_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [0]:
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

     |████████████████████████████████| 102kB 2.7MB/s 
     |████████████████████████████████| 112kB 2.6MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 614kB 46.5MB/s 
     |████████████████████████████████| 235kB 44.6MB/s 
     |████████████████████████████████| 307kB 46.8MB/s 
     |████████████████████████████████| 51kB 12.9MB/s 
     |████████████████████████████████| 215kB 48.4MB/s 
     |████████████████████████████████| 51kB 21.3MB/s 
     |████████████████████████████████| 737kB 56.7MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=f89b58851b66ae99714dc17dc5cc2a3044a9be3e434ae171226e1b15617258cc
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=ecbf60b817db707d08f34e55910af48fc1d22b4b1384c2d93451680c4735c832
  Stored 

### Important imports / format

In [0]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [0]:
# libraries and math functions
import pandas as pd
import numpy as np
import pandas_profiling
from scipy.io import arff # for loading .arff file
from scipy.stats import randint, uniform
import random

# imports for pipeline and regression
import category_encoders as ce
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.utils.multiclass import unique_labels
from ipywidgets import interact, fixed
from xgboost import XGBRegressor, XGBClassifier
from pdpbox import pdp
from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
import eli5
from eli5.sklearn import PermutationImportance
import shap

# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### Importing Data

In [0]:

url_list = ['https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/artists.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/genres.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/labels.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/reviews.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/years.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/content.csv']

artists = pd.read_csv(url_list[0])
genres = pd.read_csv(url_list[1])
labels = pd.read_csv(url_list[2])
reviews = pd.read_csv(url_list[3])
years = pd.read_csv(url_list[4])
content = pd.read_csv(url_list[5], encoding='cp1252')

artists.shape, genres.shape, labels.shape, reviews.shape, years.shape, content.shape

((18831, 2), (22680, 2), (20190, 2), (18393, 13), (19108, 2), (18393, 2))

In [0]:
years.head()

,reviewid,year
0,22703,1998.0
1,22721,2016.0
2,22659,2016.0
3,22661,2016.0
4,22725,2016.0


### Merging Dataframes

In [0]:
# Merging the dataframes
df = artists.merge(
    genres, how='left', on='reviewid').merge(
        labels, how='left', on='reviewid').merge(
            reviews, how='left', on='reviewid').merge(
                years, how='left', on='reviewid').merge(
                    content, how='left', on='reviewid'
                )
            
print(df.shape)
df.head()

(27094, 18)


,reviewid,artist_x,genre,label,title,artist_y,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,year,content
0,22703,massive attack,electronic,virgin,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,1998.0,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,krallice,metal,hathenter,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,2016.0,"Eight years, five albums, and two EPs in, the ..."
2,22659,uranium club,rock,static shock,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,2016.0,Minneapolis’ Uranium Club seem to revel in bei...
3,22659,uranium club,rock,fashionable idiots,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,2016.0,Minneapolis’ Uranium Club seem to revel in bei...
4,22661,kleenex,rock,kill rock stars,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,2016.0,Kleenex began with a crash. It transpired one ...


In [0]:
# testing to make sure encoding worked properly 
# (reviews had "" and ' in lots of places)
df['content'][0]

"“Trip-hop” eventually became a ’90s punchline, a music-press shorthand for “overhyped hotel lounge music.” But today, the much-maligned subgenre almost feels like a secret precedent. Listen to any of the canonical Bristol-scene albums of the mid-late ’90s, when the genre was starting to chafe against its boundaries, and you’d think the claustrophobic, anxious 21st century started a few years ahead of schedule. Looked at from the right angle, trip-hop\xa0is part of an unbroken chain that runs from the abrasion of ’80s post-punk to the ruminative pop-R&B-dance fusion of the moment.\xa0The best of it has aged far more gracefully (and forcefully) than anything recorded in the waning days of the record industry’s pre-filesharing monomania has any right to. Tricky rebelled against being attached at the hip to a scene he was already looking to shed and decamped for Jamaica to record a more aggressive, bristling-energy mutation of his style in ’96; the name\xa0Pre-Millennium Tension is the on

### Export CSV file

In [0]:
from google.colab import files
df.to_csv('pitchfork_reviews.csv') 
files.download('pitchfork_reviews.csv')